<a href="https://colab.research.google.com/github/ellhar3001/pokemon_fighters-/blob/main/pokemon_fighters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [160]:
import requests
import json 
import random
import pprint
import time
class Get_Pokemon:


  def __init__(self):
    
    self.pokemon = []
    self.poke_holder = []
    self.api = 'https://pokeapi.co/api/v2/pokemon/'
    self.limit = '?limit=151'
    

  def get_pokemon(self):

    ###gets pokemon data from the PokeAPI

    response = requests.get(self.api+self.limit)
    json_data = json.loads(response.text)
    results = json_data["results"]
      
    i = 0
    for i in range(0,151):
      self.pokemon.append(results[i]["name"]) 

  
  def pokemon_selection(self):

    ##player selects pokemon from first 150 pokemon
    
    self.poke_holder = []
    print("Would you like to have a random pokemon or select one?\n")
    choice = input().lower()
 
    if choice not in self.pokemon:
      self.poke_holder.append(self.pokemon[random.randrange(0,151)])
      print("\nOk, we'll select at random....")
      print("-"*20)
      print("\nYou have " + self.poke_holder[0].capitalize())
      print("-"*20)
      print('Hold on whilst we fetch your pokemons details!')

    else:
      self.poke_holder.append(choice)

      print('\n'+self.poke_holder[0].capitalize() + " is an excellent choice!")
      print("-"*20)
      print('Hold on whilst we fetch your pokemons details!')

    


In [161]:
class Player_Stats:

  ##this class holds the player details, i.e pokemon stats and moves 
  def __init__(self):
    
    self.name = '_'
    self.hp = 0
    self.attack = 0
    self.defense = 0
    self.hp = 0
    self.special_attack = 0
    self.special_defense = 0

    self.api = 'https://pokeapi.co/api/v2/'
    self.stats_data = {}
    self.moves_available = []
    self.chosen_moves = {}
    self.move_data = {}
  
  def __str__(self):
      return self.stats_data

  def get_stats(self,editor):

    ##gets pokemon specific stats

    ##gets data about pokemon
    self.name = editor
    response = requests.get(self.api+ 'pokemon/' +self.name)
    stats_list = response.json()['stats']
    moves_list = response.json()['moves']

    ##gets pokemon specific data
    for i in range(0,5):
      var = stats_list[i]['stat']['name']
      self.stats_data[var] = stats_list[i]['base_stat']
      
    
    self.hp = self.stats_data['hp']
    self.attack = self.stats_data['attack']
    self.defense = self.stats_data['defense']
    self.special_attack = self.stats_data['special-attack']
    self.special_defense = self.stats_data['special-defense']

    ##gets moves specific to pokemon
    i = 0
    while i < len(moves_list):
      self.moves_available.append(moves_list[i]['move']['name'])
      i +=1
  
  def move_decider(self):
    self.chosen_moves = {}
    self.move_data = {}
    ##selects moves at random, then gets move specific information
    randnum = random.sample(range(0,len(self.moves_available)),4)

    for i in range(0,4):
      move = self.moves_available[randnum[i]]
      move_info = requests.get(self.api+ 'move/' + move)

      self.move_data['Accuracy'] = move_info.json()['accuracy']
      self.move_data['PP'] = move_info.json()['pp']
      self.move_data['Power'] = move_info.json()['power']
      
      for key, item in self.move_data.items():
        if item is None:
          self.move_data[key] = 0
      
      self.chosen_moves[move] = self.move_data
      self.move_data = {}
      
    randnum =[]
  
  def stats_printer(self):

    ##shows player the move available and gives option to swap them

    print('\nYour ' + self.name.capitalize() + ' stats are below:\n')
    pprint.pprint(self.stats_data)
    print('\n')
    pprint.pprint(self.chosen_moves)
    print('\n')
    print("Are you happy with the above moves?\n")
  
  def generate_opponent (self,p_list):

    ##generates new computer player 
    self.name = p_list[random.randrange(0,151)]
    self.get_stats(self.name)
    self.move_decider()

  def damage_calc(self,p_move,opp_def):

    ##takes in variables to work out the damage given for a specific move
    if random.randint(0,100) < self.chosen_moves[p_move]['Accuracy']:
      return round((((2/5 + 2) * self.chosen_moves[p_move]['Power'] * self.attack/opp_def)/50 + 2)*((self.special_attack*2)/100))
    else:
      print("Attack missed!!!")
      return 0
  
  def damage_receiver(self,var):
    #takes damage fro other player and reduces hp
    self.hp = self.hp - var
    return self.hp

In [162]:
class Gameplay:

  def __init__(self):
    self.game_number = 1
  
  def starter(self,p1,p2,p_list):
    ##begins game 
    p2.generate_opponent(p_list)
    time.sleep(3)
    print("\nOk! Lets get ready to rumble!!\n")
    print("Game " + str(self.game_number) + "!\nPlayer's " + p1.name.capitalize() +': '+ str(p1.hp) +'HP' + 
          "\nVS\nComputer's " + p2.name.capitalize() +': '+ str(p2.hp) +'HP')
    
  def fighting(self,p1,p2):
    ##takes turns between players until a player has run out of HP 
    print('\nPlease choose your attacking move!\n')
    print(p1.chosen_moves.keys())
    print('\n')
    move_choice = input()
    if move_choice not in p1.chosen_moves.keys():
      print("Not a valid choice, choose again!")
      print(p1.chosen_moves.keys())
      print('\n')
      move_choice = input()
    print('\n' + p1.name.capitalize() + ' uses ' + move_choice + '\n')
    p2.damage_receiver(p1.damage_calc(move_choice,p2.defense))
    time.sleep(1)
    if p2.hp <= 0:
      print("GAME OVER!!!\n")
    else:
      computer_move = random.choice(list(p2.chosen_moves))
      print('\n'+ p2.name.capitalize()+ ' uses ' + computer_move +'!\n')
      p1.damage_receiver(p2.damage_calc(computer_move,p1.defense))
      time.sleep(1)
      if p1.hp <= 0 or p2.hp <= 0:
        print("GAME OVER!!!\n")
        time.sleep(1)
      else:
        print("At the end of that round we have:\nPlayer's " + p1.name.capitalize() +': '+ str(p1.hp) +'HP' + 
            "\nVS\nComputer's " + p2.name.capitalize() +': '+ str(p2.hp) +'HP')
    
  def game_over(self,p1,p2,p_list):
    ##situation where the computer wins 
    time.sleep(1)
    print(p2.name.capitalize() + " is the winner!!! Bad luck " + p1.name.capitalize())
    print("Would you like to start again?\n")
    self.game_number = 1
    p_list = []
    return input()

  def winner(self,p1,p2):
    ##situation where player wins
     time.sleep(1)
     print(p1.name.capitalize() + " is the winner!!! Bad luck " + p2.name.capitalize())
     print("\nIts time to move on to the next round!!!")
     self.game_number +=1

In [163]:
def player_setup():
##player selects pokemon & gets pokemon specific data from api
  pokemon_practise.pokemon_selection()
  player1.get_stats(pokemon_practise.poke_holder[0])

  ##checks player is happy with move available
  player1.move_decider()
  player1.stats_printer()
  decision = input()[0].lower()
  while decision != 'y':
    player1.move_decider()
    player1.stats_printer()
    decision = input()[0].lower()


In [164]:
##creates player classes and starts game
game = Gameplay() 
player1 = Player_Stats()
opponent = Player_Stats()
playing = True

##generates pokemon list
pokemon_practise = Get_Pokemon()
pokemon_practise.get_pokemon()

#sets player up
player_setup()

while playing:
  ##starting dialogue
  game.starter(player1, opponent,pokemon_practise.pokemon)

  ##loops moves until there is a winner
  while player1.hp > 0 and opponent.hp > 0:
    game.fighting(player1,opponent)

  ##situation if player loses, gives option to restart or quit
  if player1.hp <= 0:
    if game.game_over(player1,opponent,pokemon_practise.poke_holder)[0] == 'y':
      playing = True
      player_setup()
      continue
    else:
      print("Thanks for playing, see you soon!")
      break
  
  #situation where player wins and they progres to next round
  else:
    game.winner(player1,opponent)
    continue

Would you like to have a random pokemon or select one?

pikachu

Pikachu is an excellent choice!
--------------------
Hold on whilst we fetch your pokemons details!

Your Pikachu stats are below:

{'attack': 55,
 'defense': 40,
 'hp': 35,
 'special-attack': 50,
 'special-defense': 50}


{'play-nice': {'Accuracy': 0, 'PP': 20, 'Power': 0},
 'slam': {'Accuracy': 75, 'PP': 20, 'Power': 80},
 'strength': {'Accuracy': 100, 'PP': 15, 'Power': 80},
 'zap-cannon': {'Accuracy': 50, 'PP': 5, 'Power': 120}}


Are you happy with the above moves?

yes

Ok! Lets get ready to rumble!!

Game 1!
Player's Pikachu: 35HP
VS
Computer's Dugtrio: 35HP

Please choose your attacking move!

dict_keys(['slam', 'strength', 'play-nice', 'zap-cannon'])


slam

Pikachu uses slam


Dugtrio uses sludge-wave!

At the end of that round we have:
Player's Pikachu: 22HP
VS
Computer's Dugtrio: 29HP

Please choose your attacking move!

dict_keys(['slam', 'strength', 'play-nice', 'zap-cannon'])


slam

Pikachu uses slam

Atta